In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Dataset/vuong_train.csv')

In [3]:
# df['label'] = df['label'].replace(-1,0)

In [4]:
df

,Unnamed: 0,len,Comment,Rating
0,0,13,xôi dẻo đồ_ăn đậm vị hộp xôi được lót lá trông...,1.0
1,1,53,gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0.0
2,2,48,thời_tiết lạnh như này cả nhà rủ nhau đến lega...,1.0
3,3,59,em có đọc review thấy mng bảo trà sữa nướng đề...,0.0
4,4,93,đồ_ăn rất ngon nhà_hàng cũng rất đẹp tất_cả đề...,1.0
...,...,...,...,...
12990,12995,159,để tui gthieu cho anh_em hàng bún đậu \ chỉ mu...,0.0
12991,12996,6,đồ_ăn ổn giá_cả ko rẻ lắm,0.0
12992,12997,53,lâu rồi mới quay lại quán có thêm nhiều món mớ...,0.0
12993,12998,63,có_lẽ do khẩu_vị mỗi người mình thì ăn thấy cũ...,0.0


In [5]:
# df = df[['RevId','Comment','label','Rating']]

In [6]:
df['Comment'].isnull().value_counts()

False    12995
Name: Comment, dtype: int64

In [7]:
# df['len'].describe()

In [8]:
# list_len = df['len'].to_list()
# list_text = df['Comment'].to_list()
# list_label = df['label'].to_list()

# for i in range(len(list_len)):
#     if list_len[i] >= 256:
#         print(list_text[i])
#         tmp = list_text[i].split()
#         list_text[i] =  ' '.join(tmp[0:210]) + ' ' + ' '.join(tmp[-46:])
#         print()
#         print(list_text[i])
#         print(list_label[i])
#         print("hihihi")

# df['Comment'] = list_text

In [9]:
df = df.dropna()
df.shape

(12995, 4)

In [10]:
df['Rating'] = df['Rating'].astype(int)

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)
df_train.shape, df_test.shape

((11695, 4), (1300, 4))

In [12]:
from sklearn.utils import shuffle

df_train_x = df_train[df_train['Rating'] == 1]
df_train_y = df_train[df_train['Rating'] == 0]
print(df_train_x.shape, df_train_y.shape)
df_train_x = df_train_x.sample(7300)
df_train_y= df_train_y.sample(1200)

df_train = pd.concat([df_train_x, df_train[df_train['Rating'] == 0], df_train_y],axis = 0)

df_train = shuffle(df_train)

(7949, 4) (3746, 4)


In [13]:
# df_test = df_test[['Comment','label']]

In [14]:
df_train['Rating'].value_counts()

1    7300
0    4946
Name: Rating, dtype: int64

In [15]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import torch
import numpy
import re
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataloader import default_collate
import random
import torch.backends.cudnn as cudnn
import numpy as np

from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras import initializers
from keras import regularizers, constraints
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping

import joblib

D:\PythonFile\miniconda3\envs\Kaggle\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [16]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

cudnn.benchmark = False
cudnn.deterministic = True

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer
print(device)

cuda


In [18]:
v_phobert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
# v_phobert.embeddings.position_embeddings = nn.Embedding(1024, 768, padding_idx=1)

In [20]:
v_phobert

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [21]:
v_tokenizer

PreTrainedTokenizer(name_or_path='vinai/phobert-base', vocab_size=64000, model_max_len=256, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'})

In [22]:
MAX_LEN = 256
print("LOAD phoBERT DONE")

def get_emb_vector(input_ids):
    input_ids  = torch.tensor([input_ids]).to(torch.long)
    with torch.no_grad():
        features = phobert(input_ids.to(torch.device("cuda:0")))
    #print(features)
    emb_vecs = features[0].cpu().numpy()[0]#[1:-1]
    #print(emb_vecs)
    return emb_vecs

LOAD phoBERT DONE


In [23]:
def text2ids(text):
    # print(tokenizer.encode("<pad> nhà <pad>"))
    tkz = tokenizer.encode(text)
    return tkz

## Data Generator

In [24]:
from keras.utils.data_utils import Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, ids, labels, batch_size=16,  max_seq_len=256, feature_len=768, n_classes=18, shuffle=True):
        self.max_seq_len = max_seq_len
        self.feature_len = feature_len
        self.batch_size = batch_size
        self.labels = labels
        self.ids = ids
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, idx_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.max_seq_len, self.feature_len))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, idx in enumerate(idx_temp):
            X[i,] = get_emb_vector(self.ids[idx])
            # Store class
            y[i] = self.labels[idx]
        # X = X[:,:,:,np.newaxis]
        # y = to_categorical(y, num_classes=self.n_classes)
        # print(y.shape)
        # exit()
        return X, y

In [25]:
# train_generator = DataGenerator(ids_train, y_train, batch_size=8, n_classes=len(classes))
# valid_generator = DataGenerator(ids_test, y_test, batch_size=8, n_classes=len(classes), shuffle=False)

## Finetuning BERT

In [26]:
# def standardize_data(row):
#     # Xóa dấu chấm, phẩy, hỏi ở cuối câu
#     row = re.sub(r"[\.,\?]+$-", "", row)
#     # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
#     row = row.replace(",", " ").replace(".", " ") \
#         .replace(";", " ").replace("“", " ") \
#         .replace(":", " ").replace("”", " ") \
#         .replace('"', " ").replace("'", " ") \
#         .replace("!", " ").replace("?", " ") \
#         .replace("-", " ").replace("?", " ")
#     return row

In [27]:
from tqdm import tqdm

class Bert_fine_tuning(nn.Module):
    def __init__(self):
        super(Bert_fine_tuning, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=768, out_features = 500, bias=True).to(device),
            nn.BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=500, out_features = 2, bias=True).to(device)
        )
        self.criterior = nn.CrossEntropyLoss()
        # self.criterior = nn.BCEWithLogitsLoss()
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        param_optimizer = list(self.bert.named_parameters())
        self.optimizer =optim.AdamW(
#             [{'params': self.bert.parameters(), 'lr': 8e-5}, 
#             {'params': self.classifier.parameters(), 'lr': 1e-3}],
            [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            {'params': self.classifier.parameters(), 'lr': 1e-3}],
            lr = 2e-5
#              momentum=0.9
        )
#         self.sigmoid = nn.Sigmoid()
      
    
    def tokenize(self,v_text, label):
        v_tokenized = []
        max_len = 256 
        for i_text in v_text:
#             i_text = standardize_data(i_text)
            line = self.tokenizer.encode(i_text, max_length = 256)
            v_tokenized.append(line)

        padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
        print('padded:', padded[0])
        print('len padded:', padded.shape)
        attention_mask = numpy.where(padded == 1, 0, 1)
        print('attention mask:', attention_mask[0])
        padded = torch.tensor(padded).to(torch.long)
        print("Padd = ",padded.size())
        attention_mask = torch.tensor(attention_mask)
        
        target = torch.tensor(label) 
        print(target)
        data_tensor = TensorDataset(padded, target) 
        train_loader = DataLoader(dataset = data_tensor, batch_size = 32)
        attention_mask = DataLoader(attention_mask, batch_size = 32)
        
        return train_loader, attention_mask
    
    def forward(self, input_ids, attention_mask):
        _, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
             return_dict=False
        )
#         self.drop = nn.Dropout(p=0.2)
#         output = self.drop(pooled_output)
        output = self.classifier(output)
        return output
#         return self.sigmoid(output)
    
    

In [28]:
import gc

def train_model(bert, save_path, df_train, df_val, num_epochs=1):
        text_train = df_train['Comment'].to_list()
        label_train = df_train['Rating'].to_list()
        text_test = df_test['Comment'].to_list()
        label_test = df_test['Rating'].to_list()
        
        Data_train, attention_train = bert.tokenize(text_train, label_train)
        Data_test, attention_test = bert.tokenize(text_test, label_test)
        
        dataloader_dict = {
            'train' : Data_train,
            'test' : Data_test
        }

        list_tmp = []
        
        for epoch in range(num_epochs + 1):
            if epoch == 0:
                print()
                continue
            print("Epoch {}/{} ".format(epoch, num_epochs))

            for phase in ['train', 'test']:
                if phase == 'train':
                    bert.train()
                    attention = attention_train
                else:
                    bert.eval()
                    attention = attention_test

                epoch_loss = 0.0
                epoch_corrects = 0

                for (inputs, labels), i_attention in tqdm(zip(dataloader_dict[phase], attention)):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    i_attention = i_attention.to(device)

                    bert.optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = bert(inputs,i_attention)

                        loss = bert.criterior(outputs, labels)

                        list_tmp.append(outputs)
                        print(outputs)

                        _, preds = torch.max(outputs, 1)

                        del outputs
                        torch.cuda.empty_cache()
                        gc.collect()

                        if phase == 'train':
                            loss.backward()
                            bert.optimizer.step()

                        epoch_loss += loss.item() * inputs.size(0)
                        epoch_corrects += torch.sum(preds == labels.data)

                    # with torch.no_grad():
                    #     outputs = bert(inputs, i_attention)
                    #     loss = bert.criterior(outputs, labels)
                    #
                    #     list_tmp.append(outputs)
                    #     print(outputs)
                    #
                    #     _, preds = torch.max(outputs, 1)
                    #
                    #     del outputs
                    #     torch.cuda.empty_cache()
                    #     gc.collect()
                    #
                    #     if phase == 'train':
                    #         loss.backward()
                    #         bert.optimizer.step()
                    #
                    #     epoch_loss += loss.item() * inputs.size(0)
                    #     epoch_corrects += torch.sum(preds == labels.data)


                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)

                print("{} Loss: {:.4f}  Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
                torch.save(bert.bert.state_dict(), "./bert_model{}.pth".format(epoch))
                torch.save(bert.state_dict(), "./full_bert_model{}.pth".format(epoch))

In [29]:
# !pip install pytorch-pretrained-bert

In [30]:
# from pytorch_pretrained_bert.optimization import BertAdam

b =  Bert_fine_tuning()
train_model(b,"./", df_train, df_test, num_epochs=5)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples t

padded: [    0   946  1813   819    99  3864    30    26   328 16132    28   154
   119    99 13295    68   328  4650  1074   946  1813  4650  2809   412
   656  1171    63  2117  1074  2122    42  1357    51    26   154    11
   779   548    40  2804  1456  4650    54   412   354  1074    54   515
  3950  4311  2117  2725   281     8  2804    50    28   390  4036    54
 11685  4650  1834   203    11     2     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1 

0it [00:02, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 8.00 GiB total capacity; 7.25 GiB already allocated; 0 bytes free; 7.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
b = Bert_fine_tuning()
w = torch.load('full_bert_model1.pth')
b.load_state_dict(w)

In [ ]:
# predict

text_test = df_test['Comment'].to_list()
label_test = df_test['Rating'].to_list()

epoch_loss = 0.0
epoch_corrects = 0
net = b
net.eval()
Data_test, attention_test = b.tokenize(text_test, label_test)
attention = attention_test

list_pred = np.array([])
            
for (inputs, labels), i_attention in tqdm(zip(Data_test, attention)):
    inputs = inputs.to(device)
    labels = labels.to(device)
    i_attention = i_attention.to(device)
    
    outputs = b(inputs,i_attention)
        
    loss = b.criterior(outputs,labels)      
        
    _, preds = torch.max(outputs, 1)
    
#     print(preds.cpu().numpy())
    list_pred = np.append(list_pred,preds.cpu().numpy())
        
    epoch_loss += loss.item() * inputs.size(0)
    epoch_corrects += torch.sum(preds == labels.data)
            
epoch_loss = epoch_loss / len(Data_test.dataset)
epoch_acc = epoch_corrects.double() / len(Data_test.dataset)
            
print(" Loss: {:.4f}  Acc: {:.4f}".format( epoch_loss, epoch_acc))

In [ ]:
# torch.save(b.bert.state_dict(),"./bert_model.pth")

In [ ]:
# list_pred

print("BERT Accuracy Score -> ",accuracy_score(list_pred, label_test)*100)

print(classification_report(label_test,list_pred))

In [ ]:
# check 1

# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]
# num_train_optimization_steps = int(args.epochs*len(train_df)/args.batch_size/args.accumulation_steps)
# optimizer = AdamW(optimizer_grouped_parameters, lr=args.lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False

In [ ]:
# Attention Layer
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'W_regularizer': self.W_regularizer,
            'u_regularizer': self.u_regularizer,
            'b_regularizer': self.b_regularizer,
            'W_constraint': self.W_constraint,
            'u_constraint': self.u_constraint,
            'b_constraint': self.b_constraint,
            'bias': self.bias
        })
        return config

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

## combine with tree-model

In [ ]:
def make_bert_features(v_text):
    global phobert, sw
    v_tokenized = []
    
    max_len = 256 
    for i_text in v_text:
#         i_text = standardize_data(i_text)
        line = tokenizer.encode(i_text, max_length = 256)
        v_tokenized.append(line)

    
    # Chèn thêm số 1 vào cuối câu nếu như không đủ 256 từ
    padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    print('padded:', padded[0])
    print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = numpy.where(padded == 1, 0, 1)
    print('attention mask:', attention_mask[0])

    padded = torch.tensor(padded).to(torch.long)
    print("Padd = ",padded.size())
    attention_mask = torch.tensor(attention_mask)
    
    padded = DataLoader(padded, batch_size=32)
    attention_mask = DataLoader(attention_mask, batch_size = 32)
    check = False
    v_feats = None
    for data, i_mask in zip(padded, attention_mask):
        with torch.no_grad():
            last_hidden_states = phobert(input_ids=data.to(device), attention_mask = i_mask.to(device))
            v_features = last_hidden_states[0][:, 0, :].cpu().numpy()
            if check is False: 
                v_feats = v_features
                check = True
            else:
                v_feats = np.append(v_feats, v_features, axis = 0)
                
        print(v_feats.shape)
                                    
    print(v_feats.shape)
    return v_feats

In [ ]:


import numpy as np
tokenizer = v_tokenizer
phobert = b.bert


text_train = df_train['Comment'].to_list()
label_train = df_train['Rating'].to_list()
text_test = df_test['Comment'].to_list()
label_test = df_test['Rating'].to_list()



features_train = make_bert_features(text_train)
features_test = make_bert_features(text_test)


## BERT Keras

In [ ]:
# def create_bert():
#
#     model = Sequential()
#     model.add(embed)
#     model.add(Dropout(0.2))
#     model.add(Dense(768, use_bias=True))
#     model.add(BatchNormalizationV1(500, epsilon=1e-5, momentum=0.1))
#     model.add(ReLU())
#     model.add(Dropout(0.1))
#     model.add(Dense(500))
#     #model.add(nn.Linear(in_features=768, out_features = 500, bias=True))
#     # model.add(nn.BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device))
#     # model.add(nn.ReLU(inplace=True))
#     # model.add(nn.Dropout(p=0.1))
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     model.build(input_shape=(1,1,256,768))
#     model.summary()
#
#     return model

In [ ]:
# bert = create_bert()
# bert.built()
# bert.fit(features_train, np.array(label_train))
# predictions_bert = bert.predict(features_test)
#
# print("BERT Accuracy Score -> ",accuracy_score(predictions_bert, label_test)*100)
# print(classification_report(label_test,predictions_bert))

In [ ]:
# from sklearn.metrics import roc_auc_score
#
# step_factor = 0.02
# threshold_value = 0.2
# roc_score=0
# predicted_proba = bert.predict_proba(features_test) #probability of prediction
# while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
#     temp_thresh = threshold_value
#     predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
#     print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
#     if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
#         roc_score = roc_auc_score(label_test, predicted)
#         thrsh_score = threshold_value
#     threshold_value = threshold_value + step_factor
# print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

## LIGBM

In [ ]:
import lightgbm as lgb

lgbm = LGBMClassifier()
train = lgbm.fit(features_train,label_train)
predictions_lgbm = lgbm.predict(features_test)
# Use accuracy_score function to get the accuracy
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, label_test)*100)
print(classification_report(label_test,predictions_lgbm))

In [ ]:
from sklearn.metrics import roc_auc_score
predictions_lgbm = (lgbm.predict_proba(features_test)[:,1] >= 0.5).astype(bool)

In [ ]:
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, label_test)*100)
print(classification_report(label_test,predictions_lgbm))
print(roc_auc_score(label_test, predictions_lgbm))

In [ ]:
# from imblearn.ensemble import BalancedBaggingClassifier

# # boosting_type = 'goss', learning_rate = 0.05, n_estimators = 200,num_leaves = 20, scale_pos_weight = 0.7
# classifier = BalancedBaggingClassifier(base_estimator=LGBMClassifier(),
#                                 sampling_strategy='not majority',
#                                 replacement=False,
#                                 random_state=42)

# classifier.fit(features_train,label_train)
# predictions_bag = classifier.predict(features_test)
# # Use accuracy_score function to get the accuracy
# print("ligbm Accuracy Score -> ",accuracy_score(predictions_bag, label_test)*100)
# print(classification_report(label_test,predictions_bag))


In [ ]:
from sklearn.metrics import roc_auc_score

step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = lgbm.predict_proba(features_test) #probability of prediction
while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

In [ ]:
predictions_lgbm = lgbm.predict(features_train)
# Use accuracy_score function to get the accuracy
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, label_train)*100)

## ADA boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
clf.fit(features_train,label_train)

predictions_ada = clf.predict(features_test)
# Use accuracy_score function to get the accuracy
print("Ada Accuracy Score -> ",accuracy_score(predictions_ada, label_test)*100)
print(classification_report(label_test,predictions_ada))

joblib.dump(clf, "ada.pkl")

## Random forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(features_train,label_train)
predictions_rf = rf.predict(features_test)
# Use accuracy_score function to get the accuracy
print("rf Accuracy Score -> ",accuracy_score(predictions_rf, label_test)*100)
print(classification_report(label_test,predictions_rf))

joblib.dump(rf, "rf.pkl")

In [ ]:
predictions_rf = rf.predict(features_test)
# Use accuracy_score function to get the accuracy
print("rf Accuracy Score -> ",accuracy_score(predictions_rf, label_test)*100)
print(classification_report(label_test,predictions_rf))

In [ ]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = rf.predict_proba(features_test) #probability of prediction
while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

In [ ]:
# df_test['predicted_label'] = predictions_rf
# df_test.to_csv('test_predict.csv', encoding = 'utf-8-sig')

In [ ]:
# df_test

## XGBoost

In [ ]:
import xgboost as xgb

xgb = xgb.XGBClassifier()
xgb.fit(features_train,label_train)
y_train_predict = xgb.predict(features_train)
print("XGB Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = xgb.predict(features_test)
print("xgb Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(xgb, "xgb.pkl")

In [ ]:
print("xgb Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)

## CAT BOOST

In [ ]:
# from catboost import CatBoostClassifier
# clf = CatBoostClassifier(task_type = "GPU", iterations = 1100,depth = 15)
#
# # ,iterations=1000,learning_rate = 0.15,depth =10)
# # , ,cat_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT7 + LIST_FEAT8)
# clf.fit(features_train,label_train)
# y_train_predict = clf.predict(features_train)
# print("CBoost Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)
#
# y_test_predict = clf.predict(features_test)
# print(classification_report(label_test,y_test_predict))
#
# joblib.dump(clf, "catboost.pkl")

## SVM

In [ ]:
SVM = SVC()
SVM.fit(features_train,label_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(features_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, label_test)*100)

joblib.dump(SVM, "svm.pkl")

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression()

reg.fit(features_train,label_train)
y_train_predict = reg.predict(features_train)
print("LG Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = reg.predict(features_test)
print("LG Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(reg, "lg.pkl")

In [ ]:
predictions_vt = (reg.predict_proba(features_test)[:,1] >= 0.51).astype(bool)

print("LG Accuracy Score -> ",accuracy_score(predictions_vt, label_test)*100)
print(classification_report(label_test,predictions_vt))
print(roc_auc_score(label_test, predictions_vt))

In [ ]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = reg.predict_proba(features_test) #probability of prediction
while threshold_value <=0.8: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

## MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation = "logistic", alpha = 0.1, hidden_layer_sizes = (10, 10, 10),
                            learning_rate = "constant", max_iter = 300, random_state = 42)

mlp.fit(features_train,label_train)
y_train_predict = mlp.predict(features_train)
print("MLP Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = mlp.predict(features_test)
print("MLP Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(reg, "mlp.pkl")

## NuSVC

In [ ]:
from sklearn.svm import NuSVC

nu_svc = NuSVC(degree = 1, kernel = "linear", nu = 0.1, probability = True, max_iter=300)

nu_svc.fit(features_train,label_train)
y_train_predict = nu_svc.predict(features_train)
print("NuSVC Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = nu_svc.predict(features_test)
print("NuSVC Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(reg, "nusvc.pkl")

## LSTM

In [ ]:
#Building the model
def create_model():
    model = Sequential()
    embed = Embedding(input_dim = 60000, output_dim = 20, input_length = features_train.shape[1])
    model.add(embed)
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(200, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(200, return_sequences = True)))
    model.add(AttentionWithContext())
    model.add(Dropout(0.1))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    return model

## Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from keras.models import load_model

rf = RandomForestClassifier(n_estimators=500,min_samples_leaf=2,
                                min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                              random_state=42)
lgbm = LGBMClassifier(boosting_type = 'goss', learning_rate = 0.05, n_estimators = 200,num_leaves = 20, scale_pos_weight = 0.7)
lg = LogisticRegression(max_iter=200)
xgb = XGBClassifier(n_estimators = 200, max_depth=20,tree_method="hist")
svm = SVC(kernel='linear', probability=True, gamma=0.2, degree=1, verbose=True, tol=1e-5)
ada = AdaBoostClassifier(n_estimators=200, random_state=1, learning_rate=1e-5)
mlp = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
                            learning_rate = "constant", max_iter = 2000, random_state = 1000)
nu_svc = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

rf = joblib.load("rf.pkl")
lg = joblib.load("lg.pkl")
xgb = joblib.load("xgb.pkl")
svm = joblib.load("svm.pkl")
ada = joblib.load("ada.pkl")
mlp = joblib.load("mlp.pkl")
nu_svc = joblib.load("nusvc.pkl")

lstm = KerasClassifier(build_fn= lambda: create_model())
lstm._estimator_type = "classifier"

# bert = KerasClassifier(build_fn = lambda: create_bert())
# bert._estimator_type = 'classifier'

estimator = []
# estimator.append(('LR', 
#                   LogisticRegression(solver ='lbfgs', 
#                                      multi_class ='multinomial', 
#                                      max_iter = 200)))

estimator.append(('RF', rf))
estimator.append(('XGB', xgb))
estimator.append(('LGBM', lgbm))
# estimator.append(('MLP', mlp))
# estimator.append(('SVM', svm ))
# estimator.append(('NuSVC', nu_svc))
estimator.append(('LG', lg))
# estimator.append(('LSTM', lstm))
estimator.append(('ADA', ada))
# estimator.append(('BERT', bert))
# estimator.append(('LSTM', lstm))

# voting ='soft'
vot_hard = VotingClassifier(estimators = estimator, voting ='soft')
vot_hard.fit(features_train,label_train)
y_pred = vot_hard.predict(features_test)
                 
print("Voting Accuracy Score -> ",accuracy_score(y_pred, label_test)*100)

print(classification_report(label_test,y_pred))

In [ ]:
predictions_vt = (vot_hard.predict_proba(features_test)[:,1] >= 0.47).astype(bool)

print("voting Accuracy Score -> ",accuracy_score(predictions_vt, label_test)*100)
print(classification_report(label_test,predictions_vt))
print(roc_auc_score(label_test, predictions_vt))

In [ ]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = vot_hard.predict_proba(features_test) #probability of prediction
while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

## Stack

In [ ]:
from sklearn.ensemble import StackingClassifier

estimators = [
    # ('RF', rf),
    # ('XGB', xgb),
    # ('LGBM', lgbm),
    # ('SVM', svm),
    ("RF", rf),
    ('XGB', xgb),
    ('LGBM', lgbm),
    # ('LSTM', lstm),
    # ('NuSVC', nu_svc)
]

st = StackingClassifier(estimators=estimators, final_estimator = ada)
st.fit(features_train,label_train)
y_pred = st.predict(features_test)
                 
print("ST Accuracy Score -> ",accuracy_score(y_pred, label_test)*100)

print(classification_report(label_test,y_pred))

In [ ]:
predictions_st = (st.predict_proba(features_test)[:,1] >= 0.5).astype(bool)

print("ST Accuracy Score -> ",accuracy_score(predictions_st, label_test)*100)
print(classification_report(label_test,predictions_st))
print(roc_auc_score(label_test, predictions_st))

In [ ]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = st.predict_proba(features_test) #probability of prediction
while threshold_value <=0.8: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

In [ ]:
b

In [ ]:
df_submit = pd.read_csv('Dataset/vuong_test.csv')
select_indices = list(np.where(df_submit.loc[:,"Comment"].isnull()))
for item in select_indices[0]: #select_indices is an array, you need to access the first element to print the indexes
    print(item)
df_submit = df_submit.dropna()
df.shape

In [ ]:
text_sub = df_submit['Comment'].to_list()
features_sub = make_bert_features(text_sub)

In [ ]:
predicted_sub = vot_hard.predict_proba(features_sub)

In [ ]:
df_submit['Rating'] = predicted_sub[:,1]

In [ ]:
df_submit['Comment'].iloc[0]

In [ ]:
df_submit[['RevId','Rating']].to_csv('submit_bert.csv', index=False)

In [ ]:
# predict


net = b
net.eval()
Data_test, attention_test = b.tokenize(text_test, label_test)
attention = attention_test

list_pred = np.array([])
            
for (inputs, labels), i_attention in tqdm(zip(Data_test, attention)):
    inputs = inputs.to(device)
    labels = labels.to(device)
    i_attention = i_attention.to(device)
    
    outputs = b(inputs,i_attention)
        
    loss = b.criterior(outputs,labels)      
        
    _, preds = torch.max(outputs, 1)
    
#     print(preds.cpu().numpy())
    list_pred = np.append(list_pred,preds.cpu().numpy())
        
    epoch_loss += loss.item() * inputs.size(0)
    epoch_corrects += torch.sum(preds == labels.data)
            
epoch_loss = epoch_loss / len(Data_test.dataset)
epoch_acc = epoch_corrects.double() / len(Data_test.dataset)
            
print(" Loss: {:.4f}  Acc: {:.4f}".format( epoch_loss, epoch_acc))